In [1]:
import pandas as pd
import numpy as np
import sys
import shutil
import glob
import os
import nibabel as nib
from pathlib import PosixPath
from scipy import ndimage
import matplotlib.pyplot as plt
from pathlib import Path
import random
import json
import itertools
import scipy
from scipy import stats
from scipy.io import loadmat
from scipy import ndimage
import sys
import matplotlib.image as mpimg
import skimage
import nibabel as nib

In [62]:
def nifti_loader(filename):
    x = nib.load(filename)
    img = np.asanyarray(x.dataobj)    
    return img
'''
args:
path = /path/to/base/directory/patient_id

assumes the t2 lesion is called patient_id_t2lesion.nii.gz
assumes the cavity mask is called patient_id_synth_cav.nii.gz
'''

def generate_bidirectional_cavity_distance_map(path)
    label = path+'_bidirectional_cavity_distance_map.nii.gz'
    cav_path = path+'_synth_cav.nii.gz'
    t2l_path = path+'_t2lesion.nii.gz'
    
    #load the t2 cavity mask
    x = nib.load(str(cav_path))
    xs = np.asanyarray(x.dataobj)
    xn = np.float64(xs)
    xn = (xn>0).astype(np.float64())
    
    #load the t2 lesion mask
    y = nib.load(str(t2l_path))
    ys = np.asanyarray(y.dataobj)
    yn = np.float64(ys)
    yn = (yn>0).astype(np.float64())

    #euclidian distance transform of the t2 cavity mask
    n_dist = ndimage.distance_transform_edt(xn==0)
    
    #calculate the inverse distance to the cavity and normalize
    i_wgt = yn * n_dist
    i_wgt /= np.max(abs(i_wgt))
    
    #calcuate the proportional distance to the cavity and normalize 
    p_wgt = abs(1-i_wgt)
    p_wgt *= yn
    p_wgt /= np.max(abs(p_wgt))

    #calculate the bi-directional distance map to the cavity
    o_wgt = abs(i_wgt-p_wgt)
    o_wgt /= np.max(np.abs(o_wgt))
    
    #add 1 to mitigate vanishing gradients
    o_wgt +=1
    
    #save the distance map weightings
    ni_img = nib.Nifti1Image(o_wgt, y.affine)
    nib.save(ni_img, os.path.join(label))
